# Capítulo II: Retrieval

### Document loaders

In [4]:
#!pip install unstructured pdf2image pdfminer pdfminer-six

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.document_loaders import OnlinePDFLoader

In [3]:
#Este es un ejemplo de carga de un documento pdf que se encuentra de forma online. Existen múltiples opciones para documentos con ubicación local
loader = OnlinePDFLoader("https://portal.beneficiosestudiantiles.cl/sites/default/files/decreto-97_09-oct-2013.pdf")

In [8]:
data = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JN679YH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JN679YH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [11]:
data[0].page_content[:250]

'Decreto 97, EDUCACIÓN (2013)\n\nDecreto 97 REGLAMENTA EL PROGRAMA DE BECAS DE EDUCACIÓN SUPERIOR\n\nMINISTERIO DE EDUCACIÓN\n\nFecha Publicación: 09-OCT-2013 | Fecha Promulgación: 22-FEB-2013 Tipo Versión: Última Versión De : 02-MAR-2023\n\nUltima Modificaci'

### Document Transformer

Cuando se trata de documentos genéricos, se sugiere que esta transformación es la más adecuada. Ya que utiliza la lista ["\n\n", "\n", " ", ""], esto tiene el efecto de intentar mantener todos los párrafos (y luego las oraciones, y después las palabras) juntos el mayor tiempo posible, ya que genéricamente parecerían ser las piezas de texto semánticamente relacionadas más fuertemente.

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

In [14]:
all_splits = text_splitter.split_documents([data[0]])

In [15]:
#También es posible particionar por token, usando tiktoken
#!pip install tiktoken
from langchain.text_splitter import TokenTextSplitter

In [17]:
token_text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=10)

In [18]:
all_token_splits = token_text_splitter.split_documents([data[0]])

### Text Embedding Model

In [19]:
#En este ejercicio usaremos el modelo de OpenAI para Embeddings
from langchain.embeddings import OpenAIEmbeddings

In [20]:
#Instanciamos el modelo de Embeddings
embeddings_model = OpenAIEmbeddings()

In [21]:
#Se aplica el modelo a una muestra de 5 chunks
embeddings = embeddings_model.embed_documents(
    [x.page_content for x in all_splits[:5]]
)

In [22]:
#Dimensiones del tensor: total de vectores, largo de cada vector
len(embeddings), len(embeddings[0])

(5, 1536)

### Vector Store

Si bien es posible almacenar temporalmente los embeddings en caché, para fines de esta sesión usaremos una base vectorial. En este caso, será ChromaDB

In [23]:
from langchain.vectorstores import Chroma

In [25]:
#Construcción de la bbdd
db = Chroma.from_documents(all_token_splits, embeddings_model)

In [26]:
query = "¿Es posible acceder a alguna beca siendo del 5 quintil de ingresos?"
docs = db.similarity_search(query)

In [27]:
print(docs[0].page_content)

 Nº 20.129 al 31 de diciembre del año anterior al proceso de asignación de becas respectivo. 2. Pertenecer a los primeros siete deciles de menores ingresos de la población del país. Decreto 108, 3. Tener un buen rendimiento académico. Para determinar EDUCACIÓN esta condición


### Retrievers

Un retriever es una interfaz que devuelve documentos a partir de una consulta no estructurada. Es más general que un vector database. Un retriever no necesita poder almacenar documentos, solo devolverlos (o recuperarlos). Los vector databases pueden usarse como la base de un retriever, pero también existen otros tipos de retriever.

Para entender retrievers, usaremos el ejercicio de preguntas-respuestas sobre documentos. Las etapas generales que debe cumplir este proceso son:
* Crear un índice
* Crear un retriever desde ese índice
* Crear una cadena de pregunta-respuesta
* ¡A hacer preguntas!

In [28]:
from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA #<--- Por ahora un golazo :)
from langchain.llms import OpenAI

In [29]:
loader = OnlinePDFLoader("https://portal.beneficiosestudiantiles.cl/sites/default/files/decreto-97_09-oct-2013.pdf")
documents = loader.load()

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(documents)

In [31]:
embeddings = OpenAIEmbeddings()

In [32]:
db = Chroma.from_documents(texts, embeddings)

In [33]:
#Ya creado el indice, hacemos la interfaz para el retriever
retriever = db.as_retriever()

In [34]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever) #<--- acá se materializa el golazo

In [35]:
query = "¿Es posible acceder a alguna beca siendo del 5 quintil de ingresos?"
qa.run(query)

' No, solamente se permiten estudiantes que provengan de los primeros siete deciles de menores ingresos de la población del país.'

In [36]:
query = "Quiero acceder a gratuidad, que condiciones debo cumplir?"
qa.run(query)

' Para acceder a la gratuidad, deberás cumplir con los requisitos establecidos en el artículo 44 bis del Decreto 77 y en la ley Nº 20.129. Además, tu hogar debe pertenecer a los siete deciles de menores ingresos de la población del país y la institución de educación superior debe contar con acreditación institucional vigente de al menos cuatro años, al 31 de diciembre del año anterior a la asignación del beneficio.'

In [77]:
#Para lo anterior, LangChain tiene una solución rápida
from langchain.indexes import VectorstoreIndexCreator

In [78]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [82]:
query = "¿Es posible acceder a alguna beca siendo del 5 quintil de ingresos?"
index.query(query)

' No, solo los estudiantes que provienen de los primeros siete deciles de menores ingresos de la población del país pueden acceder a esta beca.'